In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [5]:
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Preprocess the data
data= data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [7]:
# Encode categorical variables
label_encoder_gender= LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [8]:
# One-hot encode 'Geography'
onehot_encoder_geo= OneHotEncoder(handle_unknown='ignore')
geo_encoded= onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df= pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [9]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [10]:
# Combine one-hot encoded columns with original data
data= pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
# Split the data into features and target
X= data.drop('EstimatedSalary', axis=1)
y=data['EstimatedSalary']

In [13]:
## Split the data into train testing sets
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42)

In [37]:
## Scale these features
scaler_regression= StandardScaler()
X_train= scaler_regression.fit_transform(X_train)
X_test= scaler_regression.fit_transform(X_test)

In [ ]:
# Save the encoders and scaler for later use
with open('label_encoder_gender', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)



In [38]:
with open('scaler_regression.pkl', 'wb') as file:
    pickle.dump(scaler_regression, file)

### ANN regression 

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [16]:
# Build the model
model= Sequential([
    Dense(64, activation='relu', input_shape= (X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # output layer for regression
])

## Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()

c:\Users\Hatice\Documents\GitHub\churn_modelling\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime


In [21]:

# Set up TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [22]:
tensorflow_callback=TensorBoard(log_dir=log_dir, histogram_freq=1)

In [23]:
## Set up Early Stopping
early_stopping_callback= EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [24]:
# Train the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[early_stopping_callback, tensorflow_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 100514.0391 - mae: 100514.0391 - val_loss: 98509.3594 - val_mae: 98509.3594
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 98920.8828 - mae: 98920.8828 - val_loss: 96946.5312 - val_mae: 96946.5312
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 97781.1172 - mae: 97781.1172 - val_loss: 92957.0234 - val_mae: 92957.0234
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 93041.0625 - mae: 93041.0625 - val_loss: 86350.4844 - val_mae: 86350.4844
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 86673.2500 - mae: 86673.2500 - val_loss: 77840.0469 - val_mae: 77840.0469
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 77327.4531 - mae: 77327.4531 - val_loss: 68976.1094 - val_mae: 68976.1094
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 68064.2656 - mae: 68064.2656 - val_loss: 61157.6406 - val_mae: 61157.6406
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step 

In [25]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir regressionlogs/fit

In [31]:
## Evaluate model on the test data

test_loss, test_mae= model.evaluate(X_test, y_test)
print(f'Test MAE: {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51132.4102 - mae: 51132.4102
Test MAE: 50311.921875


In [32]:
model.save('regression_model.h5')